In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("./data/heart.csv")
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import KBinsDiscretizer

features = ['age', 'trtbps', 'chol', 'thalachh', 'oldpeak', 'thall']
y_feat = 'remainder__output'
preprocessing_pipeline = make_column_transformer(
    (KBinsDiscretizer(encode="onehot", n_bins=3), features),
    remainder="passthrough",
)

In [6]:
from sklearn.model_selection import KFold
from tqdm.auto import tqdm

from neural_fca import NeuralFCA

kf = KFold(n_splits=5)
cv_f1_scores = []
fcas = []

for train, test in tqdm(kf.split(df)):
    df_train = df.iloc[train, :]
    df_test= df.iloc[test, :]

    preprocessing_pipeline.fit(df_train)
    df_train_bin = pd.DataFrame(
        preprocessing_pipeline.transform(df_train), 
        columns=preprocessing_pipeline.get_feature_names_out()
    ).astype("bool")
    df_test_bin = pd.DataFrame(
        preprocessing_pipeline.transform(df_test), 
        columns=preprocessing_pipeline.get_feature_names_out()
    ).astype("bool")

    X_train, y_train = df_train_bin.drop(y_feat, axis=1), df_train_bin[y_feat]
    X_test, y_test = df_test_bin.drop(y_feat, axis=1), df_test_bin[y_feat]

    X_train.index = X_train.index.map(str)
    X_test.index = X_test.index.map(str)

    fca = NeuralFCA().fit(X_train, y_train)
    cv_f1_scores.append(fca.score(X_test, y_test))
    fcas.append(fca)

0it [00:00, ?it/s]

/mnt/sdb1/home/l.morozov/classical-ml-entrypoint/venv/lib/python3.8/site-packages/sklearn/preprocessing/_discretization.py:322: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 4 are removed. Consider decreasing the number of bins.
  warnings.warn(
/mnt/sdb1/home/l.morozov/classical-ml-entrypoint/venv/lib/python3.8/site-packages/sklearn/preprocessing/_discretization.py:322: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 5 are removed. Consider decreasing the number of bins.
  warnings.warn(
/mnt/sdb1/home/l.morozov/classical-ml-entrypoint/venv/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of th

In [7]:
cv_f1_scores

[0.6444444444444445, 0.0, 0.8269230769230769, 0.0, 0.0]

In [8]:
np.mean(cv_f1_scores)

0.2942735042735043